In [1]:
import json  
import pandas as pd  
from pandas.io.json import json_normalize  
import requests
from tabulate import tabulate
from sklearn.cluster import KMeans
import random
import numpy as np
import pandas as pd
import folium
import warnings
warnings.filterwarnings('ignore')
import os
os.environ.keys()

KeysView(environ({'ALLUSERSPROFILE': 'C:\\ProgramData', 'APPDATA': 'C:\\Users\\HP\\AppData\\Roaming', 'COMMONPROGRAMFILES': 'C:\\Program Files\\Common Files', 'COMMONPROGRAMFILES(X86)': 'C:\\Program Files (x86)\\Common Files', 'COMMONPROGRAMW6432': 'C:\\Program Files\\Common Files', 'COMPUTERNAME': 'LAPTOP-G0R1GHT9', 'COMSPEC': 'C:\\WINDOWS\\system32\\cmd.exe', 'CONDA_DEFAULT_ENV': 'base', 'CONDA_EXE': 'D:\\Anaconda\\Scripts\\conda.exe', 'CONDA_PROMPT_MODIFIER': '(base) ', 'CONDA_PYTHON_EXE': 'D:\\Anaconda\\python.exe', 'CONDA_SHLVL': '1', 'CURL_CA_BUNDLE': 'C:\\Program Files\\PostgreSQL\\16\\ssl\\certs\\ca-bundle.crt', 'DRIVERDATA': 'C:\\Windows\\System32\\Drivers\\DriverData', 'EFC_2588': '1', 'FPS_BROWSER_APP_PROFILE_STRING': 'Internet Explorer', 'FPS_BROWSER_USER_PROFILE_STRING': 'Default', 'GDAL_DATA': 'C:\\Program Files\\PostgreSQL\\16\\gdal-data', 'HOMEDRIVE': 'C:', 'HOMEPATH': '\\Users\\HP', 'INTELLIJ IDEA COMMUNITY EDITION': 'C:\\Program Files\\JetBrains\\IntelliJ IDEA Communi

In [3]:
#Fetching data form HERE API for IIT Bombay
url = 'https://discover.search.hereapi.com/v1/discover?in=circle:19.1334,72.9133;r=10000&q=apartment&apiKey=9o4PlKsSvIFd5xmuT9ITBbtGM7buHPBT52U-UqqF0rQ'
data = requests.get(url).json()
d=json_normalize(data['items'])
d.to_csv('apartment.csv')
d

,title,id,language,ontologyId,resultType,access,distance,categories,contacts,address.label,...,address.state,address.county,address.city,address.district,address.subdistrict,address.street,address.postalCode,position.lat,position.lng,address.building
0,Sejal Enterprises,here:pds:place:356te7uv-0d474f1b290f4599a3ae10...,en-GB,here:cm:ontology:apartment_rental_flat_rental,place,"[{'lat': 19.13242, 'lng': 72.99675}]",8767,"[{'id': '700-7200-0324', 'name': 'Apartment/Fl...",[{'phone': [{'value': '+919167349689'}]}],"Sejal Enterprises, Talavali Gaon Road, Sector ...",...,Maharashtra,Thane,Navi Mumbai,Ghansoli,Sector 23,Talavali Gaon Road,400701,19.13247,72.99675,NaN
1,Parth Enclave,here:pds:place:356te7uv-415dac51001e45508db78d...,en,here:cm:ontology:apartment_rental_flat_rental,place,"[{'lat': 19.12962, 'lng': 72.99682}]",8794,"[{'id': '700-7200-0324', 'name': 'Apartment/Fl...",NaN,"Parth Enclave, Parth Enclave Street, Sector 21...",...,Maharashtra,Thane,Navi Mumbai,Ghansoli,Sector 21,Parth Enclave Street,400701,19.12973,72.99692,NaN
2,Vrisha Enterprises,here:pds:place:356te7uv-f1ba79396f0448198139c5...,en-GB,here:cm:ontology:apartment_rental_flat_rental,place,"[{'lat': 19.14125, 'lng': 72.99928}]",9089,"[{'id': '700-7200-0324', 'name': 'Apartment/Fl...",[{'phone': [{'value': '+918080106061'}]}],"Vrisha Enterprises, Zoeb Villa, Sector 29 B, R...",...,Maharashtra,Thane,Navi Mumbai,Rabale,Sector 29 B,NaN,400701,19.14122,72.99942,Zoeb Villa


In [5]:
#Cleaning API data
d2=d[['title','address.label','distance','access','position.lat','position.lng','address.postalCode','contacts','id']]
d2.to_csv('cleaned_apartment.csv')
d2

,title,address.label,distance,access,position.lat,position.lng,address.postalCode,contacts,id
0,Sejal Enterprises,"Sejal Enterprises, Talavali Gaon Road, Sector ...",8767,"[{'lat': 19.13242, 'lng': 72.99675}]",19.13247,72.99675,400701,[{'phone': [{'value': '+919167349689'}]}],here:pds:place:356te7uv-0d474f1b290f4599a3ae10...
1,Parth Enclave,"Parth Enclave, Parth Enclave Street, Sector 21...",8794,"[{'lat': 19.12962, 'lng': 72.99682}]",19.12973,72.99692,400701,NaN,here:pds:place:356te7uv-415dac51001e45508db78d...
2,Vrisha Enterprises,"Vrisha Enterprises, Zoeb Villa, Sector 29 B, R...",9089,"[{'lat': 19.14125, 'lng': 72.99928}]",19.14122,72.99942,400701,[{'phone': [{'value': '+918080106061'}]}],here:pds:place:356te7uv-f1ba79396f0448198139c5...


In [6]:
#Counting no. of cafes, department stores and gyms near apartments around IIT Bombay
df_final=d2[['position.lat','position.lng']]

CafeList=[]
ResList=[]
GymList=[]
latitudes = list(d2['position.lat'])
longitudes = list( d2['position.lng'])
for lat, lng in zip(latitudes, longitudes):    
    radius = '1000' #Set the radius to 1000 metres
    latitude=lat
    longitude=lng
    
    search_query = 'cafe' #Search for any cafes
    url = 'https://discover.search.hereapi.com/v1/discover?in=circle:{},{};r={}&q={}&apiKey=9o4PlKsSvIFd5xmuT9ITBbtGM7buHPBT52U-UqqF0rQ'.format(latitude, longitude, radius, search_query)
    results = requests.get(url).json()
    venues=json_normalize(results['items'])
    CafeList.append(venues['title'].count())
	
    search_query = 'gym' #Search for any gyms
    url = 'https://discover.search.hereapi.com/v1/discover?in=circle:{},{};r={}&q={}&apiKey=9o4PlKsSvIFd5xmuT9ITBbtGM7buHPBT52U-UqqF0rQ'.format(latitude, longitude, radius, search_query)
    results = requests.get(url).json()
    venues=json_normalize(results['items'])
    GymList.append(venues['title'].count())

    search_query = 'restaurents' #search for supermarkets
    url = 'https://discover.search.hereapi.com/v1/discover?in=circle:{},{};r={}&q={}&apiKey=9o4PlKsSvIFd5xmuT9ITBbtGM7buHPBT52U-UqqF0rQ'.format(latitude, longitude, radius, search_query)
    results = requests.get(url).json()
    venues=json_normalize(results['items'])
    ResList.append(venues['title'].count())

df_final['Cafes'] = CafeList
df_final['Restaurents'] = ResList
df_final['Gyms'] = GymList

df_final

,position.lat,position.lng,Cafes,Restaurents,Gyms
0,19.13247,72.99675,2,20,2
1,19.12973,72.99692,2,20,2
2,19.14122,72.99942,2,20,6


In [7]:
#Run K-means clustering on dataframe
kclusters = 3

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_final)
df_final['Cluster']=kmeans.labels_
df_final['Cluster']=df_final['Cluster'].apply(str)
df_final

,position.lat,position.lng,Cafes,Restaurents,Gyms,Cluster
0,19.13247,72.99675,2,20,2,0
1,19.12973,72.99692,2,20,2,2
2,19.14122,72.99942,2,20,6,1


In [8]:
#Plotting clustered locations on map using Folium

#define coordinates of the college
map_bom=folium.Map(location=[19.1334,72.9133],zoom_start=12)

# instantiate a feature group for the incidents in the dataframe
locations = folium.map.FeatureGroup()

# set color scheme for the clusters
def color_producer(cluster):
    if cluster=='0':
        return 'green'
    elif cluster=='1':
        return 'orange'
    else:
        return 'red'

latitudes = list(df_final['position.lat'])
longitudes = list(df_final['position.lng'])
labels = list(df_final['Cluster'])
names=list(d2['title'])
for lat, lng, label,names in zip(latitudes, longitudes, labels,names):
    folium.CircleMarker(
            [lat,lng],
            fill=True,
            fill_opacity=1,
            popup=folium.Popup(names, max_width = 300),
            radius=5,
            color=color_producer(label)
        ).add_to(map_bom)

# add locations to map
map_bom.add_child(locations)
folium.Marker([19.1334,72.9133],popup='IIT Bombay').add_to(map_bom)

In [9]:
#saving the map 
map_bom.save("map-IITBombay.html")